# Further Experimentation

- Standardize answers
- Cleaning points
- Standard amount of information
- Rearrange data

## I. Load Data

In [ ]:
import ujson

In [ ]:
file = open('verbalNew.json')
verbal = ujson.load(file)
file.close()

In [ ]:
verbal

In [ ]:
file = open('spatialNew.json')
spatial = ujson.load(file)
file.close()

In [ ]:
file = open("GQA/sceneGraphs/train_sceneGraphs.json")
trainImgs = ujson.load(file) # length: 74942
file.close

In [ ]:
file = open("GQA/sceneGraphs/val_sceneGraphs.json")
trainImgs.update(ujson.load(file)) # length: 74942
file.close()

## Standardize Answers

In [ ]:
# standardize answers
import re

for i in verbal:
    vKey = verbal[i]
    sKey = spatial[i]
    
    # https://www.geeksforgeeks.org/python-remove-punctuation-from-string/
    vKey['answer'] = re.sub(r'[^\w\s]', '', vKey['answer'].lower())
    sKey['answer'] = re.sub(r'[^\w\s]', '', sKey['answer'].lower())

In [ ]:
verbal

In [ ]:
len(verbal)

## Clean Points

In [ ]:
import matplotlib.pyplot as plt
from visual_genome import api as vg
from PIL import Image as PIL_Image
import requests
from io import BytesIO

In [ ]:
def visualize_points(imgID, points):
    image = vg.get_image_data(id=imgID)
    response = requests.get(image.url)
    img = PIL_Image.open(BytesIO(response.content))
    plt.imshow(img)
    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)
    ax = plt.gca()
    for o in range(len(points)):
        ax.plot(points[o]['x'],points[o]['y'], 'ro') 
        ax.text(points[o]['x'], points[o]['y'], str(o), style='italic', bbox={'facecolor':'white', 'alpha':0.7, 'pad':10})
    plt.tick_params(labelbottom='off', labelleft='off')
    plt.show()

In [ ]:
# count = 0
for i in spatial:
    qKey = spatial[i]
    imageID = qKey['imageId']
    ans = qKey['answer']
    if 'V' not in i: # GQA question
        obj = trainImgs[imageID]['objects'][qKey['objectID']]
        qKey['pt'] = {'x': int(obj['x'] + obj['w'] / 2), 'y': int(obj['y'] + obj['h'] / 2), 'answer': ans}
    else:  # visual genome question
        qKey['pt'] = qKey['points'][0] # TO DO: rename 'ans' to 'answer'

## Standardize Amount of Info and Rearrange (for Verbal, Spatial)

In [ ]:
verbalN = {}
spatialN = {}

In [ ]:
file = open('VisualGenomeSpatial.json')
vgSpatial = ujson.load(file)
file.close()

In [ ]:
for i in spatial:
    sKey = spatial[i]
    qID = i
    qFound = False
    for j in vgSpatial:
        for k in j['qas']:
            
            if "V" + str(k['qa_id']) == qID:
                sKey['question'] = k['question']
                qFound = True
                break
        if qFound:
            break

In [ ]:
# standardize amount of information
for i in verbal:
    vKey = verbal[i]
    sKey = spatial[i]
    qID = i
    imageId = int(vKey['imageId'])
    vQuest = vKey['question']
    sQuest = sKey['question']
    ans = vKey['answer']
    pt = sKey['pt']
    if imageId not in verbalN:
        verbalN[imageId] = [{'qa_id': qID, 'question': vQuest, 'answer': ans}]
        spatialN[imageId] = [{'qa_id': qID, 'question': sQuest, 'answer': ans, 'point': pt}]
    else:
        verbalN[imageId].append({'qa_id': qID, 'question': vQuest, 'answer': ans})
        spatialN[imageId].append({'qa_id': qID, 'question': sQuest, 'answer': ans, 'point': pt})

In [ ]:
count = 0
for i in spatialN:
    for k in spatialN[i]:
        count += 1
print(count)

In [ ]:
# write to new json file
with open("verbal.json", "w") as outfile: 
    ujson.dump(verbalN, outfile)

# write to new json file
with open("spatial.json", "w") as outfile: 
    ujson.dump(spatialN, outfile)

In [ ]:
def showImgPoints(imgID, o):
    img = PIL_Image.open("GQA/images/" + str(imgID) + ".jpg")
    plt.imshow(img)
    ax = plt.gca()
    ax.plot(o['x'],o['y'], 'ro') 
    ax.text(o['x'],o['y'], o['answer'], style='italic', color='red')
    plt.tick_params(labelbottom='off', labelleft='off')
    plt.show()

In [ ]:
for i in spatialN:
    for j in spatialN[i]:
        showImgPoints(i, j['point'])
        print(j['question'])
        print(j['answer'])